In [1]:
import requests
import pandas as pd
import json
import io
import os
import pymysql
from dotenv import load_dotenv
from pandas import json_normalize

load_dotenv()

globalpassw =os.environ.get('PASS')
globaldb=os.environ.get('DB')
globalapi=os.environ.get('API')

'''print(globalapi)
print(globaldb)
print(globalpassw)'''
#mindless change for Git


def ticker_job():
    sym = ""
    ticker = ["vym", "mo", "ryld"]
    api_key=globalapi

    url = "https://alpha-vantage.p.rapidapi.com/query"
    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com"
    }

    querystring = {
        "interval": "60min",
        "function": "TIME_SERIES_INTRADAY",
        "symbol": sym,
        "datatype": "csv",
        "output_size": "full",
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    CSVResponse = response.content
    df1 = pd.read_csv(io.StringIO(CSVResponse.decode("utf-8")))
    df1.insert(0, "Ticker", sym, True)

    # Create an empty DataFrame to store the final results
    finaldf = pd.DataFrame()

    # For loop to fetch data for each ticker
    for sym in ticker:
        querystring["symbol"] = sym
        response = requests.request("GET", url, headers=headers, params=querystring)
        CSVResponse = response.content
        df2 = pd.read_csv(io.StringIO(CSVResponse.decode("utf-8")))
        df2.insert(0, "Ticker", sym, True)
        finaldf = pd.concat([finaldf, df2], axis=0)

    # End of the loop for multiple runs
    

    # Reset the index of the final DataFrame
    finaldf.reset_index(drop=True, inplace=True)

    # Connect to the MariaDB database
    host = "192.168.0.46"
    port = 3306
    user = "root"
    password = globalpassw
    database = globaldb

    connection = pymysql.connect(
        host=host,
        port=port,
        user=user,
        password=password,
        database=database,
    )

    # Insert data into the existing table
    table_name = "TickerData"

    try:
        with connection.cursor() as cursor:
            
            for _, row in finaldf.iterrows():
                query = f"INSERT INTO {table_name} (Ticker, Timestamp, Open, High, Low, Close, Volume) VALUES (%s, %s, %s, %s, %s, %s, %s)"
                cursor.execute(
                    query,
                    (
                        row["Ticker"],
                        row["timestamp"],
                        row["open"],
                        row["high"],
                        row["low"],
                        row["close"],
                        row["volume"]
                    ),
                )
        connection.commit()
        print("Data inserted successfully into the table.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        connection.close()
    print("Job Ran")

ticker_job()

    

5f30e471f2msh07de88cc828fe55p13f896jsn07b920203904
VandyDB
Jinxyouow3m3asoda
Data inserted successfully into the table.
Job Ran
